<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Kompyuternoe_zrenie/Zadachi_detekcii_segmentacii/DZ_Kondratev_The_Nature_Conservancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание**

**Преподаватель:** Наталья Баданина, Юлия Пономарева, Ярослав Сапронов, Артём Качалкин, Павел Мехнин

В данном задании вам предстоит помочь [The Nature Conservancy](https://www.nature.org/en-us/about-us/where-we-work/united-states/california/) в борьбе с рыбаками-браконьерами. Необходимо по снимкам с камеры на рыболовецком судне классифицировать улов. За основу решения предлагается взять модель детектора:
[github.com...tion.ipynb](https://github.com/a4tunado/lectures/blob/master/007/007-detection.ipynb). Решение необходимо прислать в виде ссылки на ipython-ноутбук с указанием значения метрики на [Leaderboard](https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring/leaderboard).

### 1. Импорт библиотек и загрузка данных

In [ ]:
import glob
import os
import zipfile
import shutil

In [ ]:
# подключим kaggle и скачаем датасет
from google.colab import files
# нужно выбрать kaggle.json
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"stefkon82","key":"b01f82295f6caca3fd66207958de205d"}'}

In [ ]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# скачиваем датасет
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring

100% 2.10G/2.11G [00:30<00:00, 18.8MB/s]
100% 2.11G/2.11G [00:31<00:00, 72.6MB/s]


In [ ]:
!apt-get install p7zip-full -y

In [ ]:
!unzip /content/the-nature-conservancy-fisheries-monitoring.zip

In [ ]:
!unzip /content/sample_submission_stg1.csv.zip

Archive:  /content/sample_submission_stg1.csv.zip
  inflating: sample_submission_stg1.csv  


In [ ]:
!unzip /content/sample_submission_stg2.csv.zip

Archive:  /content/sample_submission_stg2.csv.zip
  inflating: sample_submission_stg2.csv  


In [ ]:
import pandas as pd

# Чтение содержимого sample_submission_stg1.csv
df1 = pd.read_csv('/content/sample_submission_stg1.csv')
print("Первые строки из sample_submission_stg1.csv:")
print(df1.head())

# Чтение содержимого sample_submission_stg2.csv
df2 = pd.read_csv('/content/sample_submission_stg2.csv')
print("\nПервые строки из sample_submission_stg2.csv:")
print(df2.head())

Первые строки из sample_submission_stg1.csv:
           image       ALB       BET       DOL       LAG       NoF     OTHER  \
0  img_00005.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   
1  img_00007.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   
2  img_00009.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   
3  img_00018.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   
4  img_00027.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   

      SHARK       YFT  
0  0.046585  0.194283  
1  0.046585  0.194283  
2  0.046585  0.194283  
3  0.046585  0.194283  
4  0.046585  0.194283  

Первые строки из sample_submission_stg2.csv:
           image       ALB       BET       DOL       LAG       NoF     OTHER  \
0  img_00005.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   
1  img_00007.jpg  0.455003  0.052938  0.030969  0.017734  0.123081  0.079142   
2  img_00009.jpg  0.455003  0.052938  0.0309

In [ ]:
!unzip -q /content/train.zip

In [ ]:
!unzip -q /content/test_stg1.zip

In [ ]:
!7z x test_stg2.7z

In [ ]:
!unzip /content/boxes.zip

In [ ]:
!apt-get install tree -y

In [ ]:
!tree -L 1

.
├── boxes
├── boxes.zip
├── kaggle.json
├── __MACOSX
├── my_model.h5
├── sample_data
├── sample_submission_stg1.csv.zip
├── sample_submission_stg2.csv.zip
├── test_stg1
├── test_stg1.zip
├── test_stg2
├── test_stg2.7z
├── the-nature-conservancy-fisheries-monitoring.zip
├── train
└── train.zip

6 directories, 9 files


In [ ]:
%ls /content/boxes

alb_labels.json  dol_labels.json  shark_labels.json
bet_labels.json  lag_labels.json  yft_labels.json


In [ ]:
%ls /content/train

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [ ]:
%ls /content/test_stg2 | head -n 10

image_00001.jpg
image_00002.jpg
image_00003.jpg
image_00004.jpg
image_00005.jpg
image_00006.jpg
image_00007.jpg
image_00008.jpg
image_00009.jpg
image_00010.jpg


### 2. Подготовка данных

In [ ]:
import json
import os

# Определим путь к директории с аннотациями
boxes_dir = '/content/boxes'

# Список классов рыб с аннотациями bounding box
fish_classes = ['alb', 'bet', 'dol', 'lag', 'shark', 'yft']

annotations = {}

# Загружаем аннотации для каждого класса
for fish in fish_classes:
    json_file = os.path.join(boxes_dir, f'{fish}_labels.json')
    with open(json_file, 'r') as f:
        annotations[fish] = json.load(f)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Определяем параметры аугментации данных
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Масштабируем пиксельные значения
    rotation_range=30,        # Случайные повороты
    width_shift_range=0.1,    # Случайные горизонтальные сдвиги
    height_shift_range=0.1,   # Случайные вертикальные сдвиги
    shear_range=0.2,          # Сдвиги
    zoom_range=0.2,           # Зум
    horizontal_flip=True,     # Случайные горизонтальные отражения
    fill_mode='nearest',      # Режим заполнения для трансформированных пикселей
    validation_split=0.2      # 20% данных для валидации
)

# Для валидационных данных только масштабирование
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
batch_size = 32
image_size = (224, 224)  # MobileNetV2 ожидает изображения размером 224x224

# Определение директории с обучающими данными
train_dir = '/content/train'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',       # Устанавливаем как обучающую выборку
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',     # Устанавливаем как валидационную выборку
    shuffle=False
)

Found 3025 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


### 3. Создание и компиляция модели

In [ ]:
# Загружаем базовую модель
base_model = MobileNetV2(
    weights='imagenet',        # Загружаем веса, предварительно обученные на ImageNet
    include_top=False,         # Не включаем классификатор
    input_shape=(224, 224, 3)
)

# Замораживаем слои базовой модели
base_model.trainable = False

# Добавляем собственные верхние слои
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(8, activation='softmax')(x)

model = Model(inputs, outputs)

# Компилируем модель
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### 4. Обучение модели

In [ ]:
# Обучение модели

epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.5004 - loss: 1.6104 - val_accuracy: 0.6037 - val_loss: 1.0605
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.6412 - loss: 0.9571 - val_accuracy: 0.6582 - val_loss: 0.9518
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.7044 - loss: 0.8126 - val_accuracy: 0.7128 - val_loss: 0.7911
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.7591 - loss: 0.7056 - val_accuracy: 0.7327 - val_loss: 0.7450
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.7668 - loss: 0.6515 - val_accuracy: 0.7846 - val_loss: 0.6144
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.8160 - loss: 0.5377 - val_accuracy: 0.8045 - val_loss: 0.6060
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.8085 - loss: 0.5410 - val_accuracy: 0.7926 - val_loss: 0.6205
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 214s 2s/step - accuracy: 0.8314 - loss: 0.4889 - val_accuracy: 0.7846 - val_loss: 0.

### Промежуточные результаты обучения

Мы наблюдаем рост точности на валидационном наборе данных, что указывает на успешное обучение модели.


In [ ]:
# Сохранение всей модели в формате .h5
model.save('my_model.h5')

In [ ]:
from google.colab import files

# Скачивание сохраненной модели
files.download('my_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import files

In [ ]:
from tensorflow.keras.models import load_model

# Загрузка заранее обученной модели
model = load_model('my_model.h5')

### 5. Предсказание на тестовых данных

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Пути к директории с тестовыми изображениями stg1
test_dir1 = '/content/test_stg1'

# Загрузка файла с образцом для первого этапа
sample_submission1 = pd.read_csv('sample_submission_stg1.csv')

# Получаем список идентификаторов изображений из образца
image_ids_stage1 = sample_submission1['image'].tolist()

# Проверка количества изображений
print(f"Количество изображений для Stage 1: {len(image_ids_stage1)}")

# Подготовка к предсказанию
image_size = (224, 224)
predictions1 = []

# Проходим по всем тестовым изображениям в порядке, указанном в sample_submission
for image_id in image_ids_stage1:
    # Получаем имя файла без пути
    image_file = os.path.basename(image_id)
    img_path = os.path.join(test_dir1, image_file)

    # Проверяем, существует ли файл
    if not os.path.exists(img_path):
        print(f"Изображение {img_path} не найдено.")
        continue

    # Загружаем и обрабатываем изображение
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Нормализация

    # Предсказание
    pred = model.predict(img_array, verbose=0)
    predictions1.append(pred[0])

# Создаем DataFrame для отправки
submission_df1 = pd.DataFrame(predictions1, columns=sample_submission1.columns[1:])  # Пропускаем первую колонку 'image'
submission_df1.insert(0, 'image', sample_submission1['image'])  # Используем колонку 'image' из эталонного файла

# Сохранение DataFrame в CSV
submission_df1.to_csv('submission_stg1.csv', index=False)

Количество изображений для Stage 1: 1000


In [ ]:
import os

# Путь к директории с тестовыми изображениями Stage 2
test_dir2 = '/content/test_stg2'

# Получаем список всех файлов в директории
image_files = os.listdir(test_dir2)

# Фильтруем только файлы с расширением .jpg (если это необходимо)
image_files = [f for f in image_files if f.lower().endswith('.jpg')]

# Проверка количества изображений
print(f"Количество изображений для Stage 2: {len(image_files)}")

Количество изображений для Stage 2: 12153


In [ ]:
# Подготовка к предсказанию
predictions2 = []
image_ids = []

# Размер изображений, который использовался при обучении модели (замените на ваш размер)
image_size = (224, 224)  # Это пример, замените на ваши значения

# Проходим по всем тестовым изображениям
for image_file in image_files:
    img_path = os.path.join(test_dir2, image_file)

    # Проверяем, что файл существует (хотя мы уже получили список существующих файлов, это на всякий случай)
    if not os.path.exists(img_path):
        print(f"Изображение {img_path} не найдено.")
        continue

    # Загружаем и обрабатываем изображение
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Нормализация

    # Предсказание
    pred = model.predict(img_array, verbose=0)
    predictions2.append(pred[0])  # Предполагается, что модель возвращает массив предсказаний
    image_ids.append(image_file)

In [ ]:
import pandas as pd

# Предполагается, что у вас есть список названий классов
class_names = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']  # Замените на ваши классы, если они отличаются

# Создаем DataFrame из предсказаний
submission_df2 = pd.DataFrame(predictions2, columns=class_names)
submission_df2.insert(0, 'image', image_ids)  # Добавляем столбец с именами изображений

# Добавляем префикс 'test_stg2/' к именам файлов в столбце 'image'
submission_df2['image'] = 'test_stg2/' + submission_df2['image']

# Смотрим на первые строки DataFrame
print(submission_df2.head())

                       image       ALB       BET       DOL           LAG  \
0  test_stg2/image_02630.jpg  0.060685  0.008218  0.000321  2.263525e-04   
1  test_stg2/image_10681.jpg  0.279469  0.000718  0.000073  1.283372e-08   
2  test_stg2/image_01756.jpg  0.078455  0.000264  0.000220  5.055735e-08   
3  test_stg2/image_03779.jpg  0.693738  0.075645  0.080729  1.578407e-04   
4  test_stg2/image_06384.jpg  0.154048  0.000663  0.000370  1.749551e-04   

        NoF     OTHER     SHARK       YFT  
0  0.168046  0.003221  0.396883  0.362399  
1  0.679361  0.000223  0.000048  0.040107  
2  0.915437  0.000162  0.000025  0.005437  
3  0.003258  0.126641  0.001953  0.017879  
4  0.103730  0.036144  0.050371  0.654501  


In [ ]:
# Сохранение DataFrame в CSV
submission_df2.to_csv('submission_stg2.csv', index=False)

In [ ]:
# Загрузка файлов предсказаний
submission_stg1 = pd.read_csv('submission_stg1.csv')
submission_stg2 = pd.read_csv('submission_stg2.csv')

# Объединяем два датафрейма
combined_submission = pd.concat([submission_stg1, submission_stg2], ignore_index=True)

# Проверяем размер нового файла
print(f"Combined Submission Shape: {combined_submission.shape}")

# Сохранение объединенного файла в CSV
combined_submission.to_csv('combined_submission.csv', index=False)

Combined Submission Shape: (13153, 9)


### 6. Финальная отправка результатов

In [ ]:
!kaggle competitions submit -c the-nature-conservancy-fisheries-monitoring -f combined_submission.csv -m "Combined submission for stage 1 and stage 2"

100% 1.55M/1.55M [00:00<00:00, 7.71MB/s]
Successfully submitted to The Nature Conservancy Fisheries Monitoring

In [ ]:
import subprocess

# Получение результатов в переменную
result = subprocess.run(["kaggle", "competitions", "submissions", "-c", "the-nature-conservancy-fisheries-monitoring"], capture_output=True, text=True)

# Разделение на строки и вывод первых
lines = result.stdout.splitlines()
print("\n".join(lines[:3]))  # Печатает первые строки

fileName                          date                 description                                  status    publicScore  privateScore  
--------------------------------  -------------------  -------------------------------------------  --------  -----------  ------------  
combined_submission.csv           2024-11-26 13:47:42  Combined submission for stage 1 and stage 2  complete  1.64284      3.29151       


### Результаты и дальнейшие шаги

Мы успешно завершили этапы обучения, валидации и предсказания для двух тестовых наборов. Предложим следующее:

1. Провести большее количество эпох для обучения, чтобы повысить точность модели.
2. Использовать более сложные стратегии аугментации, чтобы улучшить обобщающую способность модели.
3. Рассмотреть возможность использования методов регуляризации для предотвращения переобучения.

## Заключение

Созданная классификационная модель позволяет автоматизировать процесс идентификации улова и способствует борьбе с браконьерством подводных ресурсов. Благодаря работе с различными методами обработки и аугментации данных, а также дообучению предварительно обученной модели, удалось достичь удовлетворительных результатов в классификации видов рыб.